In [2]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
elif PROJECT_ROOT.name.startswith("0"):
    PROJECT_ROOT = PROJECT_ROOT.parent.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn import model_selection
import xgboost as xgb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from match_forecast.utils import *

2025-04-16 11:14:33.963 | INFO     | match_forecast.config:<module>:11 - PROJ_ROOT path is: /Users/maichoun/QRT-Challenge-2024


In [5]:
PROJ_ROOT = Path("QRT-Challenge-2024").resolve().parents[1]
DATA_DIR = PROJ_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
INTERIM_DATA_DIR = DATA_DIR / "interim"

In [6]:
try:
    train_data_teams = pd.read_csv(INTERIM_DATA_DIR / "train_data_teams.csv", index_col=0)
    train_data_players = pd.read_csv(INTERIM_DATA_DIR / "train_data_players.csv", index_col=0)
    train_scores = pd.read_csv(RAW_DATA_DIR / "Y_train.csv", index_col=0)
    print("Files loaded")
    
except FileNotFoundError as e:
    print(e)

Files loaded


In [12]:
train_data = train_data_teams.merge(train_data_players, left_index=True, right_index=True, how='inner')

In [14]:
train_data.head()

,DIFF_SHOTS_TOTAL_season_sum,DIFF_SHOTS_INSIDEBOX_season_sum,DIFF_SHOTS_OFF_TARGET_season_sum,DIFF_SHOTS_ON_TARGET_season_sum,DIFF_SHOTS_OUTSIDEBOX_season_sum,DIFF_PASSES_season_sum,DIFF_SUCCESSFUL_PASSES_season_sum,DIFF_SAVES_season_sum,DIFF_CORNERS_season_sum,DIFF_FOULS_season_sum,DIFF_YELLOWCARDS_season_sum,DIFF_REDCARDS_season_sum,DIFF_OFFSIDES_season_sum,DIFF_ATTACKS_season_sum,DIFF_PENALTIES_season_sum,DIFF_SUBSTITUTIONS_season_sum,DIFF_BALL_SAFE_season_sum,DIFF_DANGEROUS_ATTACKS_season_sum,DIFF_INJURIES_season_sum,DIFF_GOALS_season_sum,DIFF_GAME_WON_season_sum,DIFF_GAME_DRAW_season_sum,DIFF_GAME_LOST_season_sum,DIFF_SHOTS_TOTAL_season_average,DIFF_SHOTS_INSIDEBOX_season_average,DIFF_SHOTS_OFF_TARGET_season_average,DIFF_SHOTS_ON_TARGET_season_average,DIFF_SHOTS_OUTSIDEBOX_season_average,DIFF_PASSES_season_average,DIFF_SUCCESSFUL_PASSES_season_average,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_season_average,DIFF_SAVES_season_average,DIFF_CORNERS_season_average,DIFF_BALL_POSSESSION_season_average,DIFF_FOULS_season_average,DIFF_YELLOWCARDS_season_average,DIFF_REDCARDS_season_average,DIFF_OFFSIDES_season_average,DIFF_ATTACKS_season_average,DIFF_PENALTIES_season_average,DIFF_SUBSTITUTIONS_season_average,DIFF_BALL_SAFE_season_average,DIFF_DANGEROUS_ATTACKS_season_average,DIFF_INJURIES_season_average,DIFF_GOALS_season_average,DIFF_GAME_WON_season_average,DIFF_GAME_DRAW_season_average,DIFF_GAME_LOST_season_average,DIFF_SHOTS_TOTAL_season_std,DIFF_SHOTS_INSIDEBOX_season_std,DIFF_SHOTS_OFF_TARGET_season_std,DIFF_SHOTS_ON_TARGET_season_std,DIFF_SHOTS_OUTSIDEBOX_season_std,DIFF_PASSES_season_std,DIFF_SUCCESSFUL_PASSES_season_std,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_season_std,DIFF_SAVES_season_std,DIFF_CORNERS_season_std,DIFF_BALL_POSSESSION_season_std,DIFF_FOULS_season_std,DIFF_YELLOWCARDS_season_std,DIFF_REDCARDS_season_std,DIFF_OFFSIDES_season_std,DIFF_ATTACKS_season_std,DIFF_PENALTIES_season_std,DIFF_SUBSTITUTIONS_season_std,DIFF_BALL_SAFE_season_std,DIFF_DANGEROUS_ATTACKS_season_std,DIFF_INJURIES_season_std,DIFF_GOALS_season_std,DIFF_SHOTS_TOTAL_5_last_match_sum,DIFF_SHOTS_INSIDEBOX_5_last_match_sum,DIFF_SHOTS_OFF_TARGET_5_last_match_sum,DIFF_SHOTS_ON_TARGET_5_last_match_sum,DIFF_SHOTS_OUTSIDEBOX_5_last_match_sum,DIFF_PASSES_5_last_match_sum,DIFF_SUCCESSFUL_PASSES_5_last_match_sum,DIFF_SAVES_5_last_match_sum,DIFF_CORNERS_5_last_match_sum,DIFF_FOULS_5_last_match_sum,DIFF_YELLOWCARDS_5_last_match_sum,DIFF_REDCARDS_5_last_match_sum,DIFF_OFFSIDES_5_last_match_sum,DIFF_ATTACKS_5_last_match_sum,DIFF_PENALTIES_5_last_match_sum,DIFF_SUBSTITUTIONS_5_last_match_sum,DIFF_BALL_SAFE_5_last_match_sum,DIFF_DANGEROUS_ATTACKS_5_last_match_sum,DIFF_INJURIES_5_last_match_sum,DIFF_GOALS_5_last_match_sum,DIFF_GAME_WON_5_last_match_sum,DIFF_GAME_DRAW_5_last_match_sum,DIFF_GAME_LOST_5_last_match_sum,DIFF_SHOTS_TOTAL_5_last_match_average,DIFF_SHOTS_INSIDEBOX_5_last_match_average,DIFF_SHOTS_OFF_TARGET_5_last_match_average,DIFF_SHOTS_ON_TARGET_5_last_match_average,DIFF_SHOTS_OUTSIDEBOX_5_last_match_average,DIFF_PASSES_5_last_match_average,DIFF_SUCCESSFUL_PASSES_5_last_match_average,DIFF_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,DIFF_SAVES_5_last_match_average,DIFF_CORNERS_5_last_match_average,DIFF_BALL_POSSESSION_5_last_match_average,DIFF_FOULS_5_last_match_average,DIFF_YELLOWCARDS_5_last_match_average,DIFF_REDCARDS_5_last_match_average,DIFF_OFFSIDES_5_last_match_average,DIFF_ATTACKS_5_last_match_average,DIFF_PENALTIES_5_last_match_average,DIFF_SUBSTITUTIONS_5_last_match_average,DIFF_BALL_SAFE_5_last_match_average,DIFF_DANGEROUS_ATTACKS_5_last_match_average,DIFF_INJURIES_5_last_match_average,DIFF_GOALS_5_last_match_average,DIFF_GAME_WON_5_last_match_average,DIFF_GAME_DRAW_5_last_match_average,DIFF_GAME_LOST_5_last_match_average,DIFF_SHOTS_TOTAL_5_last_match_std,DIFF_SHOTS_INSIDEBOX_5_last_match_std,DIFF_SHOTS_OFF_TARGET_5_last_match_std,DIFF_SHOTS_ON_TARGET_5_last_match_std,DIFF_SHOTS_OUTSIDEBOX_5_last_match_std,DIFF_PASSES_5_last_match_std,DIFF_SUCCESSFUL